# Advanced Dataframes

In this lesson we will continue working with pandas DataFrames, and explore some more complex DataFrame manipulations.

In [1]:
import numpy as np
import pandas as pd

np.random.seed(123)

In [2]:
# Create list of values for names column.

students = ['Sally', 'Jane', 'Suzie', 'Billy', 'Ada', 'John', 'Thomas',
            'Marie', 'Albert', 'Richard', 'Isaac', 'Alan']

# Randomly generate arrays of scores for each student for each subject.
# Note that all the values need to have the same length here.

math_grades = np.random.randint(low=60, high=100, size=len(students))
english_grades = np.random.randint(low=60, high=100, size=len(students))
reading_grades = np.random.randint(low=60, high=100, size=len(students))

In [3]:
# Construct the DataFrame using the above lists and arrays.

df = pd.DataFrame({'name': students,
                   'math': math_grades,
                   'english': english_grades,
                   'reading': reading_grades,
                   'classroom': np.random.choice(['A', 'B'], len(students))})

In [4]:
df

,name,math,english,reading,classroom
0,Sally,62,85,80,A
1,Jane,88,79,67,B
2,Suzie,94,74,95,A
3,Billy,98,96,88,B
4,Ada,77,92,98,A
5,John,79,76,93,B
6,Thomas,82,64,81,A
7,Marie,93,63,90,A
8,Albert,92,62,87,A
9,Richard,69,80,94,A


## Creating Dataframes

### From Lists and Dictionaries

There are several ways to create dataframes, we've already seen how we can create a dataframe from a dictionary:

In [5]:
pd.DataFrame({'A': [1, 2, 3], 'B': [4, 5, 6]})

,A,B
0,1,4
1,2,5
2,3,6


The keys in the passed dictionary will be the column names, and the values are the data points that make up each column.

We can also create dataframes from a 2d data structure, either a numpy array or a list of lists:

In [6]:
array = np.array([[1, 2, 3], [4,5,6]])

In [7]:
array

array([[1, 2, 3],
       [4, 5, 6]])

In [9]:
pd.DataFrame(array.T, columns=['A','B'])

,A,B
0,1,4
1,2,5
2,3,6


Notice here that we had to specify the names of the columns ourselves.

### From PyDataset.

In some of the exercises, you'll need to load several datasets using the `pydataset` library. (If you get an error when trying to run the import below, use `pip` to install the `pydataset` package.) When the instructions say to load a dataset from PyDataset, you will need to do the following:

The following import is necessary to access PyDataset datasets:
```python
from pydataset import data
```

Running this code snippet will show you the valuable information doc on the dataset:
```python
data(df_string_name, show_doc=True)
```

Running this code snippet will load the dataset for use as a pandas DataFrame:
```python
df = data(df_string_name)
```

There are 757 available datasets using pydataset. Running the following code snippet in a cell will return a DataFrame with all of your options:
```python
data()
```

In [11]:
# Load the dataset and store it in the variable mpg.

from pydataset import data

In [14]:
data('df_string_name', show_doc=True)

Exception: Not valid dataset name and no similar found! Try: data() to see available.

In [15]:
data()

,dataset_id,title
0,AirPassengers,Monthly Airline Passenger Numbers 1949-1960
1,BJsales,Sales Data with Leading Indicator
2,BOD,Biochemical Oxygen Demand
3,Formaldehyde,Determination of Formaldehyde
4,HairEyeColor,Hair and Eye Color of Statistics Students
...,...,...
752,VerbAgg,Verbal Aggression item responses
753,cake,Breakage Angle of Chocolate Cakes
754,cbpp,Contagious bovine pleuropneumonia
755,grouseticks,Data on red grouse ticks from Elston et al. 2001


In [13]:

df = data(df_string_name)

NameError: name 'df_string_name' is not defined

You can view the documentation for the dataset, and any pydata dataset, by setting show_doc to True. This outputs valuable context for your dataset.

```python
data('mpg', show_doc=True) 
```

### From SQL

We can use the `read_sql` method to create a dataframe based on the results of a SQL query. To do this, we need to tell pandas how to connect to the database we are querying. The way we communicate this to pandas is with a specially formatted *connection string*.

In addition, whenever we want to connect to a database from our python code (other programming languages are similar), we will need a **driver**, a bit of software that handles the details of the database connection.

In order to connect to mysql, we'll install the `pymysql` driver packages:

`python -m pip install pymysql`

Once those are installed, we can create the connection string. In general, database connection urls will have this format:

```python
protocol://[user[:password]@]hostname/[database_name]
```

Here's an example of what one would look like:

```python
mysql+pymysql://codeup:p@assw0rd@123.123.123.123/some_db
```

Another thing we need to consider is that we don't want to publish our database credentials to github, however, we will need access to these values in our code in order to create the connection string defined above.

In order to accomplish this, we can define several variables in a file named `env.py` that contain the sensitive data, add `env.py` to our `.gitignore` file, and then import those values into another script. 

**Be 100% sure to add `env.py` to this specific repository's `.gitignore` file, even and especially, if you have already added `env.py` to your global .gitignore file. This will protect the env file for people who clone this project (like collaborators)**

In [18]:
import env

In [20]:
user = env.username
password = env.password
host = env.host
schema = 'employees'

In [25]:
conn = f'mysql+pymysql://{user}:{password}@{host}/{schema}'

In [28]:
query = 'SELECT * FROM employees LIMIT 100'

Once this url is defined, we can use it with the `read_sql` function to have pandas treat the results of a SQL query as a dataframe.

In [29]:
df = pd.read_sql(query, conn)

In [30]:
df.head()

,emp_no,birth_date,first_name,last_name,gender,hire_date
0,10001,1953-09-02,Georgi,Facello,M,1986-06-26
1,10002,1964-06-02,Bezalel,Simmel,F,1985-11-21
2,10003,1959-12-03,Parto,Bamford,M,1986-08-28
3,10004,1954-05-01,Chirstian,Koblick,M,1986-12-01
4,10005,1955-01-21,Kyoichi,Maliniak,M,1989-09-12


It is common to have longer SQL queries that we want to read into python, and an example of how we might break a query into several lines is below:

!!!danger "Passwords and Sensitive Information"
    Don't add and commit files with passwords or other sensitive information in them to a git repository!

## Exercises I

Run `python -m pip install mysqlclient pymysql` from your terminal to install pymysql and the mysqlclient.

Create a notebook or python script named `advanced_dataframes` to do your work in for these exercises.

1. Create a function named `get_db_url`. It should accept a username, hostname, password, and database name and return a url connection string formatted like in the example at the start of this lesson.

2. Use your function to obtain a connection to the `employees` database.

3. Once you have successfully run a query:

    a. Intentionally make a typo in the database url. What kind of error message do you see?
    
    b. Intentionally make an error in your SQL query. What does the error message look like?

4. Read the `employees` and `titles` tables into two separate DataFrames.

5. How many rows and columns do you have in each DataFrame? Is that what you expected?

6. Display the summary statistics for each DataFrame.

7. How many unique titles are in the `titles` DataFrame?

8. What is the oldest date in the `to_date` column? 

9. What is the most recent date in the `to_date` column?